In [17]:
#import library
import tensorflow
import pandas as pd
import numpy as np
#from google.colab import drive
import os
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE
import pickle
np.random.seed(16)
#drive.mount('/content/drive')

In [18]:
#preprocess dataset
app_dir = pd.read_csv('C:/Users/Daffa Ajiputra/Downloads/Final_Project_Dataset/Data Preparation/Application_history.csv') #path to your folder with multiple csv
cred_dir = pd.read_csv('C:/Users/Daffa Ajiputra/Downloads/Final_Project_Dataset/Data Preparation/Credit_history.csv')
data_test = pd.read_csv("C:/Users/Daffa Ajiputra/Downloads/Final_Project_Dataset/Final_Project_Dataset/data_test.csv")

app_dir = app_dir.drop_duplicates(subset='Id_customer')

data = pd.DataFrame.merge(app_dir, cred_dir, on='Id_customer')

In [19]:
data.head(10)

,Id_customer,JK,KepemilikanMobil,KepemilikanProperti,JmlAnak,Pendapatan,TipePendapatan,TingkatPendidikan,StatusKeluarga,TipeRumah,FlagMobile,FlagWorkPhone,FlagPhone,Email,Pekerjaan,JmlAnggotaKeluarga,Age,Experience,LastCreditStatus
0,5008804.0,Laki-laki,Ya,Ya,0.0,427500.0,Bekerja,Post Graduate,Menikah,Sewa Apartment,1.0,1.0,0.0,0.0,NaN,2.0,33.0,12.0,Tidak memiliki pinjaman
1,5008805.0,Laki-laki,Ya,Ya,0.0,427500.0,Bekerja,Post Graduate,Menikah,Sewa Apartment,1.0,1.0,0.0,0.0,NaN,2.0,33.0,12.0,Tidak memiliki pinjaman
2,5008806.0,Laki-laki,Ya,Ya,0.0,112500.0,Bekerja,Graduate,Menikah,Rumah Pribadi,1.0,0.0,0.0,0.0,Security staff,2.0,59.0,3.0,Tidak memiliki pinjaman
3,5008808.0,Perempuan,Tidak,Ya,0.0,270000.0,Asosiasi komersial,Graduate,Belum Menikah,Rumah Pribadi,1.0,0.0,1.0,1.0,Sales staff,1.0,52.0,8.0,Tidak memiliki pinjaman
4,5008809.0,Perempuan,Tidak,Ya,0.0,270000.0,Asosiasi komersial,Graduate,Belum Menikah,Rumah Pribadi,1.0,0.0,1.0,1.0,Sales staff,1.0,52.0,8.0,Tidak memiliki pinjaman
5,5008810.0,Perempuan,Tidak,Ya,0.0,270000.0,Asosiasi komersial,Graduate,Belum Menikah,Rumah Pribadi,1.0,0.0,1.0,1.0,Sales staff,1.0,52.0,8.0,Tidak memiliki pinjaman
6,5008811.0,Perempuan,Tidak,Ya,0.0,270000.0,Asosiasi komersial,Graduate,Belum Menikah,Rumah Pribadi,1.0,0.0,1.0,1.0,Sales staff,1.0,52.0,8.0,Tidak memiliki pinjaman
7,5008812.0,Perempuan,Tidak,Ya,0.0,283500.0,Pensioner,Post Graduate,Cerai,Rumah Pribadi,1.0,0.0,0.0,0.0,NaN,1.0,62.0,1001.0,Tidak memiliki pinjaman
8,5008813.0,Perempuan,Tidak,Ya,0.0,283500.0,Pensioner,Post Graduate,Cerai,Rumah Pribadi,1.0,0.0,0.0,0.0,NaN,1.0,62.0,1001.0,Tidak memiliki pinjaman
9,5008814.0,Perempuan,Tidak,Ya,0.0,283500.0,Pensioner,Post Graduate,Cerai,Rumah Pribadi,1.0,0.0,0.0,0.0,NaN,1.0,62.0,1001.0,Tidak memiliki pinjaman


In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36257 entries, 0 to 36256
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Id_customer          36257 non-null  float64
 1   JK                   36257 non-null  object 
 2   KepemilikanMobil     36257 non-null  object 
 3   KepemilikanProperti  36257 non-null  object 
 4   JmlAnak              36257 non-null  float64
 5   Pendapatan           36257 non-null  float64
 6   TipePendapatan       36257 non-null  object 
 7   TingkatPendidikan    36257 non-null  object 
 8   StatusKeluarga       36257 non-null  object 
 9   TipeRumah            36257 non-null  object 
 10  FlagMobile           36257 non-null  float64
 11  FlagWorkPhone        36257 non-null  float64
 12  FlagPhone            36257 non-null  float64
 13  Email                36257 non-null  float64
 14  Pekerjaan            24993 non-null  object 
 15  JmlAnggotaKeluarga   36257 non-null 

In [21]:
data.isnull().sum()

Id_customer                0
JK                         0
KepemilikanMobil           0
KepemilikanProperti        0
JmlAnak                    0
Pendapatan                 0
TipePendapatan             0
TingkatPendidikan          0
StatusKeluarga             0
TipeRumah                  0
FlagMobile                 0
FlagWorkPhone              0
FlagPhone                  0
Email                      0
Pekerjaan              11264
JmlAnggotaKeluarga         0
Age                        0
Experience                 0
LastCreditStatus           0
dtype: int64

In [22]:
data.Pekerjaan.unique()

array([nan, 'Security staff', 'Sales staff', 'Accountants', 'Laborers',
       'Managers', 'Drivers', 'Core staff', 'High skill tech staff',
       'Cleaning staff', 'Private service staff', 'Cooking staff',
       'Low-skill Laborers', 'Medicine staff', 'Secretaries',
       'Waiters/barmen staff', 'HR staff', 'Realty agents', 'IT staff'],
      dtype=object)

In [23]:
#Fill nan with a value
data[data.Pekerjaan.isna()]
data.Pekerjaan = data.Pekerjaan.fillna('-')
data

,Id_customer,JK,KepemilikanMobil,KepemilikanProperti,JmlAnak,Pendapatan,TipePendapatan,TingkatPendidikan,StatusKeluarga,TipeRumah,FlagMobile,FlagWorkPhone,FlagPhone,Email,Pekerjaan,JmlAnggotaKeluarga,Age,Experience,LastCreditStatus
0,5008804.0,Laki-laki,Ya,Ya,0.0,427500.0,Bekerja,Post Graduate,Menikah,Sewa Apartment,1.0,1.0,0.0,0.0,-,2.0,33.0,12.0,Tidak memiliki pinjaman
1,5008805.0,Laki-laki,Ya,Ya,0.0,427500.0,Bekerja,Post Graduate,Menikah,Sewa Apartment,1.0,1.0,0.0,0.0,-,2.0,33.0,12.0,Tidak memiliki pinjaman
2,5008806.0,Laki-laki,Ya,Ya,0.0,112500.0,Bekerja,Graduate,Menikah,Rumah Pribadi,1.0,0.0,0.0,0.0,Security staff,2.0,59.0,3.0,Tidak memiliki pinjaman
3,5008808.0,Perempuan,Tidak,Ya,0.0,270000.0,Asosiasi komersial,Graduate,Belum Menikah,Rumah Pribadi,1.0,0.0,1.0,1.0,Sales staff,1.0,52.0,8.0,Tidak memiliki pinjaman
4,5008809.0,Perempuan,Tidak,Ya,0.0,270000.0,Asosiasi komersial,Graduate,Belum Menikah,Rumah Pribadi,1.0,0.0,1.0,1.0,Sales staff,1.0,52.0,8.0,Tidak memiliki pinjaman
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36252,5150482.0,Perempuan,Ya,Ya,1.0,157500.0,Bekerja,Graduate,Menikah,Rumah Pribadi,1.0,0.0,0.0,0.0,Core staff,3.0,30.0,5.0,0
36253,5150483.0,Perempuan,Ya,Ya,1.0,157500.0,Bekerja,Graduate,Menikah,Rumah Pribadi,1.0,0.0,0.0,0.0,Core staff,3.0,30.0,5.0,Tidak memiliki pinjaman
36254,5150484.0,Perempuan,Ya,Ya,1.0,157500.0,Bekerja,Graduate,Menikah,Rumah Pribadi,1.0,0.0,0.0,0.0,Core staff,3.0,30.0,5.0,0
36255,5150485.0,Perempuan,Ya,Ya,1.0,157500.0,Bekerja,Graduate,Menikah,Rumah Pribadi,1.0,0.0,0.0,0.0,Core staff,3.0,30.0,5.0,0


In [24]:
data = data.drop(data[data.LastCreditStatus=="Tidak memiliki pinjaman"].index)
data['LastCreditStatus'] = pd.to_numeric(data['LastCreditStatus'])
data['LastCreditStatus'] = data['LastCreditStatus'].apply(lambda x: 1 if x >= 1 else x)
data['LastCreditStatus'].value_counts()

0    14424
1     2281
Name: LastCreditStatus, dtype: int64

In [25]:
le = preprocessing.LabelEncoder()

In [26]:
cols = data.select_dtypes(include=['object']).columns.tolist()
data[cols] = data[cols].apply(le.fit_transform)
data

,Id_customer,JK,KepemilikanMobil,KepemilikanProperti,JmlAnak,Pendapatan,TipePendapatan,TingkatPendidikan,StatusKeluarga,TipeRumah,FlagMobile,FlagWorkPhone,FlagPhone,Email,Pekerjaan,JmlAnggotaKeluarga,Age,Experience,LastCreditStatus
10,5008815.0,0,1,1,0.0,270000.0,1,1,2,3,1.0,1.0,1.0,1.0,1,2.0,46.0,2.0,0
16,5008824.0,0,1,1,0.0,135000.0,0,0,2,3,1.0,0.0,0.0,0.0,9,2.0,49.0,3.0,0
17,5008825.0,1,1,0,0.0,130500.0,1,0,2,3,1.0,0.0,0.0,0.0,1,2.0,29.0,3.0,1
18,5008826.0,1,1,0,0.0,130500.0,1,0,2,3,1.0,0.0,0.0,0.0,1,2.0,29.0,3.0,1
19,5008827.0,0,1,1,0.0,180000.0,4,1,2,3,1.0,0.0,0.0,0.0,0,2.0,51.0,1001.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36250,5150480.0,1,1,1,1.0,157500.0,1,0,2,3,1.0,0.0,0.0,0.0,4,3.0,30.0,5.0,0
36252,5150482.0,1,1,1,1.0,157500.0,1,0,2,3,1.0,0.0,0.0,0.0,4,3.0,30.0,5.0,0
36254,5150484.0,1,1,1,1.0,157500.0,1,0,2,3,1.0,0.0,0.0,0.0,4,3.0,30.0,5.0,0
36255,5150485.0,1,1,1,1.0,157500.0,1,0,2,3,1.0,0.0,0.0,0.0,4,3.0,30.0,5.0,0


In [41]:
dataset = data.values
X = dataset[:,0:18]
Y = dataset[:,18]
oversample = SMOTE(sampling_strategy=0.8)
X_over, Y_over = oversample.fit_resample(X, Y)

X_train, X_test, Y_train, Y_test = train_test_split(X_over, Y_over, test_size=0.2, random_state=42)

dt = DecisionTreeClassifier(criterion='entropy', random_state=42)
dt.fit(X_train, Y_train)

DecisionTreeClassifier(criterion='entropy', random_state=42)

In [42]:
y_pred_train = dt.predict(X_train)
accuracy = metrics.accuracy_score(Y_train, y_pred_train)
print("Accuracy: {:.2f}".format(accuracy))
cm=confusion_matrix(Y_train,y_pred_train)
print('Confusion Matrix: \n', cm)
print(classification_report(Y_train, y_pred_train))

Accuracy: 1.00
Confusion Matrix: 
 [[11552    13]
 [   45  9160]]
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     11565
         1.0       1.00      1.00      1.00      9205

    accuracy                           1.00     20770
   macro avg       1.00      1.00      1.00     20770
weighted avg       1.00      1.00      1.00     20770



In [43]:
y_pred_test = dt.predict(X_test)
accuracy = metrics.accuracy_score(Y_test, y_pred_test)
print("Accuracy: {:.2f}".format(accuracy))
cm=confusion_matrix(Y_test,y_pred_test)
print('Confusion Matrix: \n', cm)
print(classification_report(Y_test, y_pred_test))

Accuracy: 0.90
Confusion Matrix: 
 [[2604  255]
 [ 266 2068]]
              precision    recall  f1-score   support

         0.0       0.91      0.91      0.91      2859
         1.0       0.89      0.89      0.89      2334

    accuracy                           0.90      5193
   macro avg       0.90      0.90      0.90      5193
weighted avg       0.90      0.90      0.90      5193



In [30]:
filename = 'decisiontree_model.sav'
pickle.dump(dt, open(filename, 'wb'))

In [34]:
model = tensorflow.keras.models.Sequential([
    tensorflow.keras.layers.Dense(128, input_dim=18, activation='relu'),
    tensorflow.keras.layers.Dense(64, activation='relu'),
    tensorflow.keras.layers.Dense(32, activation='relu'),
    tensorflow.keras.layers.Dense(18, activation='relu'),
    tensorflow.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=500, batch_size=32, verbose=0)

# evaluate the model
scores = model.evaluate(X_train, Y_train, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("tensorflow_model.h5")
print("Saved model to disk")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               2432      
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dense_6 (Dense)             (None, 18)                594       
                                                                 
 dense_7 (Dense)             (None, 1)                 19        
                                                                 
Total params: 13,381
Trainable params: 13,381
Non-trainable params: 0
_________________________________________________________________
accuracy: 55.68%
Saved model to disk


In [32]:
cols_test = data_test.select_dtypes(include=['object']).columns.tolist()
data_test[cols_test] = data_test[cols_test].apply(le.fit_transform)

#Prediction with decision tree model
load_model = pickle.load(open(filename, 'rb'))
predicted = load_model.predict(data_test)

result = pd.DataFrame({"Id_customer": data_test['Id_customer']})
predicted = predicted.astype('int64')
result['PredictedCreditStatus'] = predicted
result.to_csv('Submission_DecisionTree.csv')

c:\Program Files\Python38\lib\site-packages\sklearn\base.py:402: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


In [35]:
# load json and create model
json_file = open('C:/Users/Daffa Ajiputra/Downloads/Final_Project_Dataset/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = tensorflow.keras.models.model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("C:/Users/Daffa Ajiputra/Downloads/Final_Project_Dataset/tensorflow_model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
data_test_dup = pd.DataFrame.copy(data_test)
#data_test = data_test.drop(['FlagMobile', 'FlagWorkPhone', 'FlagPhone', 'Email'], axis=1)
data_test

Loaded model from disk


,Id_customer,JK,KepemilikanMobil,KepemilikanProperti,JmlAnak,Pendapatan,TipePendapatan,TingkatPendidikan,StatusKeluarga,TipeRumah,FlagMobile,FlagWorkPhone,FlagPhone,Email,Pekerjaan,JmlAnggotaKeluarga,Age,Experience
0,5142248,1,0,1,0,225000.0,2,0,2,3,1,0,0,0,9,2.0,54.0,6.0
1,5036925,1,1,1,0,157500.0,0,0,2,3,1,1,1,1,3,2.0,33.0,8.0
2,5126080,1,0,1,1,112500.0,2,0,2,3,1,0,0,0,7,3.0,41.0,7.0
3,5088887,1,0,1,0,171000.0,1,0,0,4,1,0,0,0,6,1.0,46.0,2.0
4,5022156,1,1,1,2,180000.0,0,1,2,3,1,0,1,1,13,4.0,32.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,5105368,0,0,1,0,360000.0,1,0,2,3,1,1,0,0,6,2.0,50.0,29.0
196,5116026,1,0,1,0,135000.0,3,0,2,3,1,0,0,0,13,2.0,56.0,1001.0
197,5067627,1,0,0,1,67500.0,1,0,2,3,1,0,0,0,6,3.0,45.0,19.0
198,5090052,0,1,1,0,270000.0,0,1,2,3,1,0,0,0,7,2.0,58.0,5.0


In [37]:
prediction = loaded_model.predict(data_test)
result = pd.DataFrame({"Id_customer": data_test_dup['Id_customer']})
prediction = prediction.astype('int64')
result['PredictedCreditStatus'] = prediction
result.to_csv('Submission_DeepLearning.csv')